In [4]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle yacs timm einops munch -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
# upload kaggle.json first.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# !apt update -qq
# !apt install -qq unzip
!kaggle datasets download nirmalsankalana/sugarcane-leaf-disease-dataset
!unzip -q sugarcane-leaf-disease-dataset.zip -d data

Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/sugarcane-leaf-disease-dataset
License(s): CC0-1.0


In [3]:
import os
import shutil
import pandas as pd

# Define paths
data_root = "data"
images_dir = os.path.join(data_root, "images")

# Create images directory if it doesn't exist
os.makedirs(images_dir, exist_ok=True)

# List to store image paths and labels
dataset = []

# Loop through each subfolder
for subfolder in os.listdir(data_root):
    subfolder_path = os.path.join(data_root, subfolder)
    
    # Ensure it's a directory
    if os.path.isdir(subfolder_path) and subfolder != "images":
        # Loop through images inside the subfolder
        for image in os.listdir(subfolder_path):
            old_image_path = os.path.join(subfolder_path, image)
            
            # Ensure it's a file (image)
            if os.path.isfile(old_image_path):
                # Define new image path in "data/images" directory
                new_image_path = os.path.join(images_dir, image)
                
                # If filename already exists, rename it to avoid conflicts
                if os.path.exists(new_image_path):
                    base, ext = os.path.splitext(image)
                    counter = 1
                    while os.path.exists(new_image_path):
                        new_image_path = os.path.join(images_dir, f"{base}_{counter}{ext}")
                        counter += 1
                
                # Move image
                shutil.move(old_image_path, new_image_path)

                # Append to dataset with updated path and original label
                dataset.append({"image_path": new_image_path, "label": subfolder})

        # Optionally remove empty subfolder after moving images
        os.rmdir(subfolder_path)

df = pd.DataFrame(dataset)
df = df.rename(columns={'image_path':'image_id'})
df["image_id"] = df["image_id"].str.replace("data/images/", "", regex=False)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

df.to_csv(os.path.join(data_root, "dataset.csv"), index=False)

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [1]:
import pandas as pd
from dataset import build_dataset, build_loader

path = 'data/images'
df = pd.read_csv('data/dataset.csv')

trd, testd, vald = build_dataset(df, path)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_loader, test_loader, val_loader = build_loader(trd, vald, 16)

In [3]:
from train import main
model = main(train_loader, val_loader, 'tswinf_nirmalsankana.pth')

2025-04-27 20:16:36.664907: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 20:16:36.678329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745784996.693916    6150 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745784996.698964    6150 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 20:16:36.715207: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[2025-04-27 20:16:38 CSWin_BOAT_64_24322_small_224](train.py 71): INFO Creating model:cswin_boat/CSWin_BOAT_64_24322_small_224


/home/ubuntu/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


[2025-04-27 20:16:39 CSWin_BOAT_64_24322_small_224](train.py 80): INFO number of params: 60235181
[2025-04-27 20:16:39 CSWin_BOAT_64_24322_small_224](train.py 101): INFO Start training
[2025-04-27 20:20:58 CSWin_BOAT_64_24322_small_224](train.py 205): INFO Train: [0/35]	eta 0:00:02 lr 0.000041	time 1.8111 (2.0567)	loss 1.5894 (1.5964)	grad_norm 2.3020 (1.5026)	mem 6408MB
[2025-04-27 20:20:58 CSWin_BOAT_64_24322_small_224](train.py 221): INFO EPOCH 0 training takes 0:04:19
[2025-04-27 20:22:05 CSWin_BOAT_64_24322_small_224](train.py 310): INFO TEST:	Time 2.339 (2.177)	Loss 1.5844 (1.5953)	Acc@1 31.250 (21.573)	Acc@5 100.000 (100.000)	Mem 6408MB
[2025-04-27 20:22:05 CSWin_BOAT_64_24322_small_224](train.py 318): INFO  * Acc@1 21.573 Acc@5 100.000
[2025-04-27 20:22:05 CSWin_BOAT_64_24322_small_224](train.py 322): INFO 
Classification Report:
              precision    recall  f1-score   support

           0     0.2081    0.8762    0.3364       105
           1     0.5294    0.1000    0.16

/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-04-27 20:40:38 CSWin_BOAT_64_24322_small_224](train.py 205): INFO Train: [3/35]	eta 0:00:02 lr 0.000160	time 1.5396 (2.4346)	loss 1.6324 (1.5422)	grad_norm 2.0478 (3.4702)	mem 6408MB
[2025-04-27 20:40:38 CSWin_BOAT_64_24322_small_224](train.py 221): INFO EPOCH 3 training takes 0:05:06
[2025-04-27 20:41:17 CSWin_BOAT_64_24322_small_224](train.py 310): INFO TEST:	Time 1.430 (1.268)	Loss 1.5234 (1.6050)	Acc@1 31.250 (24.395)	Acc@5 100.000 (100.000)	Mem 6408MB
[2025-04-27 20:41:17 CSWin_BOAT_64_24322_small_224](train.py 318): INFO  * Acc@1 24.395 Acc@5 100.000
[2025-04-27 20:41:17 CSWin_BOAT_64_24322_small_224](train.py 322): INFO 
Classification Report:
              precision    recall  f1-score   support

           0     0.2312    0.4135    0.2966       104
           1     0.3333    0.1556    0.2121        90
           2     0.0000    0.0000    0.0000       102
           3     0.2209    0.5644    0.3175       101
           4     0.7000    0.0707    0.1284        99

    accur

KeyboardInterrupt: 

In [6]:
df

,image_id,label
0,redrot (142).jpeg,2
1,redrot (311).jpeg,2
2,redrot (24).jpeg,2
3,redrot (446).jpeg,2
4,redrot (235).jpeg,2
...,...,...
2516,yellow (409).jpeg,4
2517,yellow (344).jpeg,4
2518,yellow (24).jpeg,4
2519,yellow (435).jpeg,4
